# Running trajectory 

The target of this file is to provide step by step implementation on how to launch a visualisation of a given trajectory on H1 robot.
The trajectory is given on a **.npy** file. It is supposed to perform a **squat movement**.

I had some issues with joint order, which was different that the one defined on the urdf.


In [3]:
# Imports robot and initialize visualisation
import pinocchio as pin
from pinocchio.visualize import MeshcatVisualizer
import numpy as np
import meshcat

# import robot_loader from scripts
from scripts.robot_loader import load_robot_paths

# load paths from scripts
urdf_path, mesh_path = load_robot_paths()

print("Launching MeshCat...")
viz = meshcat.Visualizer()
print("Adding a grid to represent the ground.")
viz["/Grid"].set_property("visible", True)

# Load robot into Pinocchio
print("Loading robot model into Pinocchio...")
try:
    model, collision_model, visual_model = pin.buildModelsFromUrdf(
        urdf_path,
        mesh_path,
    )
    print("Pinocchio model loaded successfully.")
except Exception as e:
    print(f"Error loading robot: {e}")

# Create Pinocchio visualizer
robot_visualizer = MeshcatVisualizer(model, collision_model, visual_model)
    
print("--- Loading complete. Objects are ready to be used. ---")

if model:
    viewer_window = viz.open()
    robot_visualizer.initViewer(viewer=viz)
    robot_visualizer.loadViewerModel()
    print("\nEnvironment is ready !")

--- Searching for robot paths ---
Detected project root directory: /home/cpene/Documents/robot_playground
Asset paths verified.
Detected urdf file: /home/cpene/Documents/robot_playground/robot_models/biped_assets/biped_assets/models/h12/h12_12dof.urdf
Detected mesh directory: /home/cpene/Documents/robot_playground/robot_models/biped_assets/biped_assets/models/h12
Launching MeshCat...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7004/static/
Adding a grid to represent the ground.
Loading robot model into Pinocchio...
Pinocchio model loaded successfully.
--- Loading complete. Objects are ready to be used. ---

Environment is ready !


In [5]:
# Display the robot on in initial position, with his feets close to the ground
q0 = pin.neutral(model)
q0[2] = 1.03

robot_visualizer.display(q0)

In [6]:
# Load trajectory
import time
import os

# Path to datas 
trajectory_file = os.path.join("data", "h1v2_squat_fixed_ankle_states.npy")

try:
    # Load data
    trajectory_data = np.load(trajectory_file)
    print(f"\nTrajectory load from '{trajectory_file}'.")
    print(f"Original dimensions : {trajectory_data.shape}")

    # The result is (176, 1, 67). That means we got one unexpected dimendion of size 1  
    # We delete this dimension (TODO : what is it supposed to be ?)
    trajectory_data = np.squeeze(trajectory_data)
    print(f"Dimensions are now : {trajectory_data.shape}")
    
except FileNotFoundError:
    print(f"ERROR : the file {trajectory_file} was not found.")
    raise


Trajectory load from 'data/h1v2_squat_fixed_ankle_states.npy'.
Original dimensions : (176, 1, 67)
Dimensions are now : (176, 67)


In [7]:
# At some point, I needed to know the dimension of the urdf. After checking,
# some modification on the urdf were needed to match dimension from the trajectory
print(f"Robot : {model.name}")
print(f"robot.nq = {model.nq}.")
print(f"robot.nv = {model.nv}.")

expected_state_dim = model.nq + model.nv

if expected_state_dim != trajectory_data.shape[1]:
    print("\n" + "="*50)
    print("ERROR : dimensions doesn't match !")
    print(f"Robot dimensions (q+v) are {expected_state_dim} ({model.nq} + {model.nv}).")
    print(f"Trajectory dimensions are {trajectory_data.shape[1]} columns.")
    print("="*50)
    raise ValueError("Dimensions doesn't match")
else:
    print("\nDimensions of both trajectory and robot matches. Great !")   

Robot : h1_5
robot.nq = 34.
robot.nv = 33.

Dimensions of both trajectory and robot matches. Great !


In [8]:
# Last problem was that joint order was not the same in the urdf and trajectory
pinocchio_order = [
    'left_hip_yaw_joint', 'left_hip_pitch_joint', 'left_hip_roll_joint', 
    'left_knee_joint', 'left_ankle_pitch_joint', 'left_ankle_roll_joint', 
    'right_hip_yaw_joint', 'right_hip_pitch_joint', 'right_hip_roll_joint', 
    'right_knee_joint', 'right_ankle_pitch_joint', 'right_ankle_roll_joint', 
    'torso_joint', 
    'left_shoulder_pitch_joint', 'left_shoulder_roll_joint', 'left_shoulder_yaw_joint', 
    'left_elbow_joint', 'left_wrist_roll_joint', 'left_wrist_pitch_joint', 'left_wrist_yaw_joint', 
    'right_shoulder_pitch_joint', 'right_shoulder_roll_joint', 'right_shoulder_yaw_joint', 
    'right_elbow_joint', 'right_wrist_roll_joint', 'right_wrist_pitch_joint', 'right_wrist_yaw_joint'
]

trajectory_file_order = [
    'left_hip_yaw_joint', 'right_hip_yaw_joint', 'torso_joint', 
    'left_hip_pitch_joint', 'right_hip_pitch_joint', 'left_shoulder_pitch_joint', 
    'right_shoulder_pitch_joint', 'left_hip_roll_joint', 'right_hip_roll_joint', 
    'left_shoulder_roll_joint', 'right_shoulder_roll_joint', 'left_knee_joint', 
    'right_knee_joint', 'left_shoulder_yaw_joint', 'right_shoulder_yaw_joint', 
    'left_ankle_pitch_joint', 'right_ankle_pitch_joint', 'left_elbow_joint', 
    'right_elbow_joint', 'left_ankle_roll_joint', 'right_ankle_roll_joint', 
    'left_wrist_roll_joint', 'right_wrist_roll_joint', 'left_wrist_pitch_joint', 
    'right_wrist_pitch_joint', 'left_wrist_yaw_joint', 'right_wrist_yaw_joint'
]

# 2. Populate a list for reodonnancement
# For each articulation in Pinocchio, we find its index on thetrajectory file.
source_indices = [trajectory_file_order.index(joint_name) for joint_name in pinocchio_order]

In [13]:
# Launch the simulation
# import converter tools from scripts
from scripts.conversion import build_pinocchio_state

print("Starting animation...")

for raw_state in trajectory_data:

    # We call the conversion tool
    q, v = build_pinocchio_state(raw_state, source_indices)

    # for Debug
    # print(f"g finale size: {q.shape[0]} (Expected: {model.nq})")

    # Send q to robot visualization
    robot_visualizer.display(q)

    # dt is 0.033333333 s
    time.sleep(0.033)

print("\nAnimation end.")

Starting animation...

Animation end.
